# 1. Split Dataset

In [1]:
from torchvision.datasets import ImageFolder            
from torch.utils.data import DataLoader, random_split

def split_data_set(dataset, random_seed):
    if random_seed is not None:
        torch.manual_seed(random_seed)  # set input as random seed

    # split image set ---> half for calibration data set, half for test data set
    dataset_length = len(dataset)
    calib_length = dataset_length // 2               
    test_length = dataset_length - calib_length      

    calib_dataset, test_dataset = random_split(dataset, [calib_length, test_length])
    return calib_dataset, test_dataset

# 2. Conformal Score

In [2]:
import numpy as np

# conformal function s(x,y)
def conformal_scores(model, dataloader, alpha=0.1, lambda_=0.1):
    scores = []  # conformal scores of image sets
    labels = []  # true label sets 
    with torch.no_grad():
        for images, true_labels in dataloader:
            images, true_labels = images.to(device), true_labels.to(device)
            outputs = model(images)
            softmaxs = torch.softmax(outputs, dim=1)

            # extract true lables' ranking/positions
            sorted_softmax, sorted_indices = torch.sort(softmaxs, descending=True, dim=1)
            true_label_positions = (sorted_indices == true_labels.unsqueeze(1)).nonzero(as_tuple=True)[1]

            # extract maximal probabilities
            max_softmax = sorted_softmax[:, 0]

            # random variable u(s)
            u = torch.rand(true_labels.size(0), device=device)
            # scores of samples whose correct label is top-ranking --> u * max_softmax
            is_top = (true_label_positions == 0)
            scores_top_rank = u * max_softmax

            # scores of samples whose correct label is  not top-ranking
            # s = max_softmax + (o-2+u) * lambda = max_softmax + (true_lable_position+1-2+u) * lambda
            scores_other_rank = max_softmax + ((true_label_positions - 1).float() + u) * lambda_
            
            conformal_scores = torch.where(is_top, scores_top_rank, scores_other_rank)
            scores.extend(conformal_scores.cpu().tolist())
            labels.extend(true_labels.cpu().tolist())
    return scores, labels

# 3. Construct Prediction Sets

In [3]:
def saps_classification(model, dataloader, t_cal, lambda_=0.1):
    saps = []         # probability set
    saps_labels = []  # label set indicated to the probability set
    labels = []       # true label
    with torch.no_grad():
        for images, true_labels in dataloader:
            images, true_labels = images.to(device), true_labels.to(device)
            outputs = model(images)
            softmax = torch.softmax(outputs, dim=1)

            # sort probabilities
            sorted_softmax, sorted_indices = torch.sort(softmax, descending=True, dim=1)

            # random variable u(s) 
            u = torch.rand(sorted_softmax.shape, device=device) # Shape: (batch_size, 10)
            # random variable for maximal probabilities
            u_f_max = torch.rand(sorted_softmax.shape[0], device=device).unsqueeze(1)       # Shape: (batch_size, 1)

            # rank of current sorted probability: [1,2,3,...,10]
            rank = torch.arange(1, sorted_softmax.size(1) + 1, device=device).unsqueeze(0)  # shape: [1, 10]

            # s = f_max + (o-2+u) * lambda
            # scores --> all the label has been calculate as non-top-ranked label now
            f_max = sorted_softmax[:, 0].unsqueeze(1)               # Shape: (batch_size, 1)
            scores = f_max + ((rank - 2).float() +  u) * lambda_    # Shape: (batch_size, 10)

            # replace the firt column with u * f_max
            scores[:, 0] = (u_f_max * f_max).squeeze(1) # Shape: (batch_size,)

            # construct prediction sets
            for i in range(len(images)):
                # select indices whose scores <= t_cal
                selected_indices = (scores[i] <= t_cal).nonzero(as_tuple=True)[0]
                
                # add selected label to prediction set
                saps.append(sorted_softmax[i][selected_indices].tolist())
                saps_labels.append(sorted_indices[i][selected_indices].tolist())
                labels.append(true_labels[i].item())  
    return saps, saps_labels, labels

# 4. Evaluate 

In [4]:
def eval_aps(aps_labels,  true_labels):
    total_set_size = 0
    coveraged = 0
    for aps_label, true_label in zip(aps_labels, true_labels):
        # cumulate total set size
        total_set_size += len(aps_label)
        # cumulate the predictions sets if it contains true label
        if true_label in aps_label:
            coveraged += 1

    # calculate average values
    samples_amount = len(true_labels)
    average_set_size = total_set_size / samples_amount
    average_coverage = coveraged / samples_amount
    return average_set_size, average_coverage

# 5. Tempreture Scaling

In [5]:
import torch
from torch import nn, optim
from torch.nn import functional as F

class ModelWithTemperature(nn.Module):
    def __init__(self, model, tempreture):
        super(ModelWithTemperature, self).__init__()
        self.model = model
        self.tempreture = tempreture
        self.temperature = nn.Parameter(torch.ones(1) * tempreture)

    def forward(self, input):
        logits = self.model(input)
        return self.temperature_scale(logits)

    def temperature_scale(self, logits):
        """
        Perform temperature scaling on logits
        """
        # Expand temperature to match the size of logits
        temperature = self.temperature.unsqueeze(1).expand(logits.size(0), logits.size(1))
        return logits / temperature

    # This function probably should live outside of this class, but whatever
    def set_temperature(self, valid_loader):
        self.cuda()
        nll_criterion = nn.CrossEntropyLoss().cuda()
        ece_criterion = _ECELoss().cuda()

        # First: collect all the logits and labels for the validation set
        logits_list = []
        labels_list = []
        with torch.no_grad():
            for input, label in valid_loader:
                input = input.cuda()
                logits = self.model(input)
                logits_list.append(logits)
                labels_list.append(label)
            logits = torch.cat(logits_list).cuda()
            labels = torch.cat(labels_list).cuda()

        # Calculate NLL and ECE before temperature scaling
        before_temperature_nll = nll_criterion(logits, labels).item()
        before_temperature_ece = ece_criterion(logits, labels).item()
        print('Before temperature - NLL: %.3f, ECE: %.3f' % (before_temperature_nll, before_temperature_ece))

        # Next: optimize the temperature w.r.t. NLL
        optimizer = optim.LBFGS([self.temperature], lr=0.01, max_iter=50)

        def eval():
            optimizer.zero_grad()
            loss = nll_criterion(self.temperature_scale(logits), labels)
            loss.backward()
            return loss
        optimizer.step(eval)

        # Calculate NLL and ECE after temperature scaling
        after_temperature_nll = nll_criterion(self.temperature_scale(logits), labels).item()
        after_temperature_ece = ece_criterion(self.temperature_scale(logits), labels).item()
        print('Optimal temperature: %.3f' % self.temperature.item())
        print('After temperature - NLL: %.3f, ECE: %.3f' % (after_temperature_nll, after_temperature_ece))

        return self


class _ECELoss(nn.Module):
    def __init__(self, n_bins=15):
        """
        n_bins (int): number of confidence interval bins
        """
        super(_ECELoss, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels):
        softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)

        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

        return ece

# 6. Optimization

In [6]:
def lambda_optimization(model, dataset, lambda_values):
    set_sizes = []
    valid_lambdas = []
    
    for current_lambda in lambda_values:
        avg_set_sizes = []
        avg_coverages = []
        
        for i in range (10):
            # run RAPS
            calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)
            calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
            calib_scores, _ = conformal_scores(model, calib_loader, 0.1, current_lambda)
            t_cal = np.quantile(calib_scores, 1 - 0.1)  
            aps, aps_labels, true_labels = saps_classification(model, test_loader, t_cal, current_lambda)
            avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)

            avg_set_sizes.append(avg_set_size)
            avg_coverages.append(avg_coverage)

        mean_set_size = np.mean(avg_set_sizes)
        mean_coverage = np.mean(avg_coverages)
        # select valid lambda with coverage guarantee
        if mean_coverage >= 0.89 :
            set_sizes.append(mean_set_size)
            valid_lambdas.append(current_lambda)

    if len(set_sizes) > 0:
        # optimal lambda has the minimal set size
        optimal_lambda_index = np.argmin(set_sizes)
        optimal_lambda = valid_lambdas[optimal_lambda_index]  
    else:
        optimal_lambda = None  # No lambda with valid coverage guarantee

    return optimal_lambda

# 7. Hyperparameter Optimization

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR10        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import detectors
import timm

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet50_cifar10", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])  # normalize
])
# load images from CIFAR10
dataset = CIFAR10(root="./data", train=False, download=True, transform=data_transform)

# tempreture scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, tempreture=5.0).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

lambda_values = [0, 1e-4, 1e-3, 0.01, 0.02, 0.05, 0.2, 0.5, 0.7, 1.0, 1.4, 2.0]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization(model, dataset, lambda_values)
if optimal_lambda == None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")

C:\Users\jiayang\anaconda3\envs\pytorch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
Before temperature - NLL: 0.357, ECE: 0.045
Optimal temperature: 4.908
After temperature - NLL: 0.829, ECE: 0.422
Looking for optimal lambda...
Optimal lambda is 2.0

